In [ ]:
import uproot
import awkward as ak
from coffea import nanoevents
from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.analysis_tools import Weights, PackedSelection
from coffea.nanoevents.methods import nanoaod
from coffea.nanoevents.methods import vector
from coffea.lookup_tools.dense_lookup import dense_lookup

ak.behavior.update(vector.behavior)

import pickle, json, gzip
import numpy as np

from typing import Optional, List, Dict
from copy import copy

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from tqdm import tqdm

# import fastjet

# import jetnet

import os

import corrections
import correctionlib

import utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
plot_dir = "../../../plots/ScaleFactors/Nov23"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
events = nanoevents.NanoEventsFactory.from_root(
    # "/eos/uscms/store/user/lpcpfnano/cmantill/v2_3/2017/HH_gen/GluGluToHHTobbVV_node_cHHH1_TuneCP5_13TeV-powheg-pythia8/GluGluToHHTobbVV_node_cHHH1/221017_221918/0000/nano_mc2017_100.root",
    "/eos/uscms/store/user/lpcpfnano/drankin/v2_2/2017/TTbar/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/TTToSemiLeptonic_ext1/211112_132937/0000/nano_mc2017_100.root",
    # "../../../../data/nano_files/v2_3/2018/TTToSemiLeptonic/nano_mc2018_101.root",
    # "../../../../data/nano_files/v2_3/2018/QCD_HT700to1000/nano_mc2018_10.root",
    # "root://cmsxrootd.fnal.gov///store/user/lpcpfnano/cmantill/v2_3/2018/TTbar/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/TTToSemiLeptonic/220808_151244/0000/nano_mc2018_1-196.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

### Boosted Top Pre-selection

Based on selection in https://indico.cern.ch/event/1208247/#10-lund-plane-reweighting-for

In [ ]:
def pad_val(
    arr: ak.Array,
    target: int,
    value: float,
    axis: int = 0,
    to_numpy: bool = True,
    clip: bool = True,
):
    """
    pads awkward array up to ``target`` index along axis ``axis`` with value ``value``,
    optionally converts to numpy array
    """
    ret = ak.fill_none(ak.pad_none(arr, target, axis=axis, clip=clip), value, axis=axis)
    return ret.to_numpy() if to_numpy else ret


def add_selection(
    name: str,
    sel: np.ndarray,
    selection: PackedSelection,
    cutflow: dict = None,
    isData: bool = False,
    signGenWeights: ak.Array = None,
):
    """adds selection to PackedSelection object and the cutflow dictionary"""
    selection.add(name, sel)
    if cutflow is not None:
        cutflow[name] = (
            np.sum(selection.all(*selection.names))
            if isData
            # add up sign of genWeights for MC
            else np.sum(signGenWeights[selection.all(*selection.names)])
        )


presel_events = events
# presel_events = events[preselection_cut]

In [ ]:
isData = False
signGenWeights = None if isData else np.sign(presel_events["genWeight"])
n_events = len(presel_events) if isData else int(np.sum(signGenWeights))
selection = PackedSelection()
weights = Weights(len(events), storeIndividual=True)
year = "2018"

cutflow = {}
cutflow["presel"] = len(presel_events)
selection_args = (selection, cutflow, isData, signGenWeights)

In [ ]:
MU_PDGID = 13

muon_selection = {
    "Id": "tight",
    "pt": 60,
    "eta": 2.4,
    "miniPFRelIso_all": 0.1,
    "dxy": 0.2,
    "count": 1,
    "delta_trigObj": 0.15,
}

ak8_jet_selection = {
    "pt": 200,
    "msd": [50, 250],
    "eta": 2.5,
    "delta_phi_muon": 2,
    "jetId": nanoaod.FatJet.TIGHT,
}

btagWPs = {"2016APV": 0.6001, "2016": 0.5847, "2017": 0.4506, "2018": 0.4168}

ak4_jet_selection = {
    "pt": 30,  # from JME-18-002
    "eta": 2.4,
    "delta_phi_muon": 2,
    "jetId": "tight",
    "puId": 4,  # loose pileup ID
    "btagWP": btagWPs,
    "ht": 250,
    "num": 2,
    "closest_muon_dr": 0.4,
    "closest_muon_ptrel": 25,
}

met_selection = {"pt": 50}

lepW_selection = {"pt": 100}

In [ ]:
muon = events.Muon
# 1024 - Mu50 trigger (https://algomez.web.cern.ch/algomez/testWeb/PFnano_content_v02.html#TrigObj)
trigObj_muon = events.TrigObj[(events.TrigObj.id == MU_PDGID) * (events.TrigObj.filterBits >= 1024)]

muon_selector = (
    (muon[f"{muon_selection['Id']}Id"])
    * (muon.pt > muon_selection["pt"])
    * (np.abs(muon.eta) < muon_selection["eta"])
    * (muon.miniPFRelIso_all < muon_selection["miniPFRelIso_all"])
    * (np.abs(muon.dxy) < muon_selection["dxy"])
)

muon_selector = muon_selector * (
    ak.count(events.Muon.pt[muon_selector], axis=1) == muon_selection["count"]
)
muon = ak.pad_none(muon[muon_selector], 1, axis=1)[:, 0]

muon_selector = ak.any(muon_selector, axis=1)
muon_selector = muon_selector * ak.any(
    np.abs(muon.delta_r(trigObj_muon)) <= muon_selection["delta_trigObj"],
    axis=1,
)

add_selection("muon", muon_selector, *selection_args)

In [ ]:
# met
met = events.MET
met_sel = met.pt >= met_selection["pt"]

# metfilters = np.ones(len(events), dtype="bool")
# metfilterkey = "data" if isData else "mc"
# for mf in self.metfilters[year][metfilterkey]:
#     if mf in events.Flag.fields:
#         metfilters = metfilters & events.Flag[mf]

# add_selection("met", met_selection * metfilters, *selection_args)
add_selection("met", met_sel, *selection_args)

# leptonic W selection
add_selection("lepW", (met + muon).pt >= lepW_selection["pt"], *selection_args)
# add_selection("lepW", met.pt + muon.pt >= 100, *selection_args)

In [ ]:
ak4_jets = events.Jet

ak4_jet_selector_no_btag = (
    ak4_jets.isTight
    # pileup ID should only be applied for pT < 50 jets (https://twiki.cern.ch/twiki/bin/view/CMS/PileupJetIDUL)
    * ((ak4_jets.puId % 2 == 1) + (ak4_jets.pt >= 50))
    * (ak4_jets.pt > ak4_jet_selection["pt"])
    * (np.abs(ak4_jets.eta) < ak4_jet_selection["eta"])
    * (np.abs(ak4_jets.delta_phi(muon)) < ak4_jet_selection["delta_phi_muon"])
)

ak4_jet_selector = ak4_jet_selector_no_btag * (
    ak4_jets.btagDeepFlavB > ak4_jet_selection["btagWP"][year]
)

ak4_selection = (
    (ak.any(ak4_jet_selector, axis=1))
    * (ak.sum(ak4_jet_selector_no_btag, axis=1) >= ak4_jet_selection["num"])
    * (ak.sum(ak4_jets[ak4_jet_selector].pt, axis=1) >= 250)
)

# add_selection("ak4_jet", ak.any(ak4_jet_selector, axis=1), *selection_args)

In [ ]:
# pileup ID effect

# unmatched
np.sum(top_match_dict["unmatched"] * ak4_selection[select])
print(166 / 170)

# w matched
np.sum(top_match_dict["w_matched"] * ak4_selection[select])
print(10 / 10)

# top matched
np.sum(top_match_dict["top_matched"] * ak4_selection[select])
print(21 / 21)

# qcd (removing muon selection, otherwise there are no events)
print(6762 / 7168)

In [ ]:
sel = selection.all(*selection.names)
muon_j_dr = muon.delta_r(events.Jet[events.Jet.isTight])
min_muon_j_dr = ak.min(muon_j_dr, axis=1)

In [ ]:
jecs = {
    "JES": "JES_jes",
    "JER": "JER",
}

jec_vars = ["pt"]  # vars we're saving that are affected by JECs

fatjets, jec_shifted_vars = corrections.get_jec_jets(events, "2018", False, jecs)
# jmsr_shifted_vars = corrections.get_jmsr(fatjets, )

In [ ]:
num_jets = 2

pt_cuts = []

for pts in jec_shifted_vars["pt"].values():
    pt_cut = np.prod(
        pad_val(
            (pts > ak8_jet_selection["pt"]),
            num_jets,
            False,
            axis=1,
        ),
        axis=1,
    )
    pt_cuts.append(pt_cut)

pt_cut = np.any(pt_cuts, axis=0)

In [ ]:
jmsValues = {}
jmrValues = {}

# https://github.com/cms-nanoAOD/nanoAOD-tools/blob/959c9ffb084bc974fb26ba2db41e3369cee04ae7/python/postprocessing/modules/jme/jetmetHelperRun2.py#L85-L110

# jet mass resolution: https://twiki.cern.ch/twiki/bin/view/CMS/JetWtagging
# nominal, down, up (these are switched in the github!!!)
jmrValues["msoftdrop"] = {
    "2016": [1.0, 0.8, 1.2],
    "2017": [1.09, 1.04, 1.14],
    # Use 2017 values for 2018 until 2018 are released
    "2018": [1.09, 1.04, 1.14],
}

# jet mass scale
# W-tagging PUPPI softdrop JMS values: https://twiki.cern.ch/twiki/bin/view/CMS/JetWtagging
# 2016 values
jmsValues["msoftdrop"] = {
    "2016": [1.00, 0.9906, 1.0094],  # nominal, down, up
    "2017": [0.982, 0.978, 0.986],
    # Use 2017 values for 2018 until 2018 are released
    "2018": [0.982, 0.978, 0.986],
}

# https://github.com/cmantill/NanoNN/blob/6bd117357e2d7ec66866b5f74790e747411efcad/python/producers/hh4bProducer.py#L154-L159

# nominal, down, up
jmrValues["particleNet_mass"] = {
    "2016": [1.028, 1.007, 1.063],
    "2017": [1.026, 1.009, 1.059],
    "2018": [1.031, 1.006, 1.075],
}
jmsValues["particleNet_mass"] = {
    "2016": [1.00, 0.998, 1.002],
    "2017": [1.002, 0.996, 1.008],
    "2018": [0.994, 0.993, 1.001],
}

In [ ]:
jmsr_vars = ["msoftdrop", "particleNet_mass"]
year = "2018"

jmsr_shifted_vars = {}

for mkey in jmsr_vars:
    tdict = {}

    mass = utils.pad_val(fatjets[mkey], num_jets, axis=1)

    np.random.seed(42)
    smearing = np.random.normal(size=mass.shape)
    # scale to JMR nom, down, up (minimum at 0)
    jmr_nom, jmr_down, jmr_up = [
        (smearing * max(jmrValues[mkey][year][i] - 1, 0) + 1) for i in range(3)
    ]
    jms_nom, jms_down, jms_up = jmsValues[mkey][year]

    mass_jms = mass * jms_nom
    mass_jmr = mass * jmr_nom

    tdict[""] = mass_jms * jmr_nom
    tdict["JMS_down"] = mass_jmr * jms_down
    tdict["JMS_up"] = mass_jmr * jms_up
    tdict["JMR_down"] = mass_jms * jmr_down
    tdict["JMR_up"] = mass_jms * jmr_up

    jmsr_shifted_vars[mkey] = tdict

In [ ]:
_ = plt.hist(jmr_nom.reshape(-1), np.linspace(0, 2, 101), histtype="step")
_ = plt.hist(jmr_up.reshape(-1), np.linspace(0, 2, 101), histtype="step")
_ = plt.hist(jmr_down.reshape(-1), np.linspace(0, 2, 101), histtype="step")

In [ ]:
_ = plt.hist(mass[mass > 30], np.linspace(0, 300, 101), histtype="step", label="Pre")
_ = plt.hist(tdict[""][mass > 30], np.linspace(0, 300, 101), histtype="step", label="Post")
_ = plt.hist(
    tdict["JMR_up"][mass > 30], np.linspace(0, 300, 101), histtype="step", label="Post JMR Up"
)
_ = plt.hist(
    tdict["JMR_down"][mass > 30], np.linspace(0, 300, 101), histtype="step", label="Post JMR Down"
)
_ = plt.hist(
    (mass * jms_nom)[mass > 30], np.linspace(0, 300, 101), histtype="step", label="Post JMS"
)
plt.legend()

In [ ]:
mkey = "msd"
jmsr_vars = ["msd", "particleNet_mass"]
year = "2018"

jms_nom, jms_down, jms_up = jmsValues[mkey][year]

In [ ]:
fatjets = events.FatJet  # CHANGE TO JEC JETS
# fatjets = corrections.get_jec_jets(events, "2018")

num_jets = 1

fatjet_selector = (
    (fatjets.pt > ak8_jet_selection["pt"])
    # * (fatjets.msoftdrop > ak8_jet_selection["msd"][0])
    # * (fatjets.msoftdrop < ak8_jet_selection["msd"][1])
    * (np.abs(fatjets.eta) < ak8_jet_selection["eta"])
    # * (np.abs(fatjets.delta_phi(muon)) > ak8_jet_selection["delta_phi_muon"])
    * (fatjets.jetId > ak8_jet_selection["jetId"])
)

leading_fatjets = ak.pad_none(fatjets[fatjet_selector], num_jets, axis=1)[:, :num_jets]
fatjet_idx = ak.argmax(fatjet_selector, axis=1)  # gets first index which is true
fatjet_selector = ak.any(fatjet_selector, axis=1)

In [ ]:
hem_region = {"eta": [-3.2, -1.3], "phi": [-0.87, -1.57]}
hem_region_selector = (
    (fatjets.eta > hem_region["eta"][0])
    * (fatjets.eta < hem_region["eta"][1])
    * (fatjets.phi > hem_region["phi"][0])
    * (fatjets.phi < hem_region["phi"][1])
)

In [ ]:
ak4_jets = events.Jet
# ak4_jets = ak.flatten(
#     ak.pad_none(events.Jet[ak.argsort(events.Jet.btagDeepB, axis=1)[:, -1:]], 1, axis=1)
# )

ak4_jet_selector = (
    (ak4_jets.jetId > ak4_jet_selection["jetId"])
    * (ak4_jets.puId >= ak4_jet_selection["puId"])
    * (ak4_jets.pt > ak4_jet_selection["pt"])
    * (np.abs(ak4_jets.eta) < ak4_jet_selection["eta"])
    * (np.abs(ak4_jets.delta_phi(muon)) < ak4_jet_selection["delta_phi_muon"])
    * (ak4_jets.btagDeepB > ak4_jet_selection["btagWP"]["2018"])
)

ak4_jet_selector = ak.any(ak4_jet_selector, axis=1)

In [ ]:
# select = ak.fill_none(muon_selector * fatjet_selector * ak4_jet_selector, False)
select = ak.fill_none(muon_selector * fatjet_selector, False)
presel_events = events[select]
leading_fatjets = leading_fatjets[select]
fatjet_idx = fatjet_idx[select]

In [ ]:
import json, gzip

with gzip.open(corrections.get_pog_json("jec", "2017"), "r") as fin:
    jec = json.loads(fin.read().decode("utf-8"))

for corr in jec["corrections"]:
    print(corr["name"])

In [ ]:
with gzip.open(corrections.get_pog_json("electron", "2017"), "r") as fin:
    egm = json.loads(fin.read().decode("utf-8"))

In [ ]:
for corr in egm["corrections"]:
    print(corr["name"])

In [ ]:
with gzip.open("../data/jec_compiled.pkl.gz", "rb") as filehandler:
    jmestuff = pickle.load(filehandler)

jmestuff["jet_factory"]

### Top matching

In [ ]:
d_PDGID = 1
b_PDGID = 5
g_PDGID = 21
TOP_PDGID = 6

ELE_PDGID = 11
vELE_PDGID = 12
MU_PDGID = 13
vMU_PDGID = 14
TAU_PDGID = 15
vTAU_PDGID = 16

Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25

b_PDGIDS = [511, 521, 523]

GRAV_PDGID = 39

GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

FILL_NONE_VALUE = -99999


skim_vars = {
    "eta": "eta",
    "phi": "phi",
    "mass": "mass",
    "pt": "pt",
}

# finding the two gen tops
presel_events = events[select]
tops = presel_events.GenPart[
    (abs(presel_events.GenPart.pdgId) == TOP_PDGID) * presel_events.GenPart.hasFlags(GEN_FLAGS)
]

In [ ]:
deltaR = 0.8

In [ ]:
tops_children = tops.distinctChildren
tops_children = tops_children[tops_children.hasFlags(GEN_FLAGS)]
ws = ak.flatten(tops_children[np.abs(tops_children.pdgId) == W_PDGID], axis=2)
had_top_sel = np.all(np.abs(ws.children.pdgId) <= 5, axis=2)
had_ws = ak.flatten(ws[had_top_sel])
had_tops = ak.flatten(tops[had_top_sel])
had_top_children = ak.flatten(tops_children[had_top_sel], axis=1)
had_bs = had_top_children[np.abs(had_top_children.pdgId) == 5]
# add_selection(
#     "hadronic bs", np.any(had_bs.pdgId, axis=1), selection, cutflow, isData, signGenWeights
# )

In [ ]:
had_w_jet_match = ak.fill_none(
    ak.all(had_ws.children.delta_r(leading_fatjets[:, 0]) < deltaR, axis=1), False
)
had_b_jet_match = ak.flatten(
    pad_val(had_bs.delta_r(leading_fatjets[:, 0]) < deltaR, 1, False, axis=1, to_numpy=False)
)
merged_top_jet_match = had_w_jet_match * had_b_jet_match

In [ ]:
top_match_dict = {
    "top_matched": had_w_jet_match * had_b_jet_match,
    "w_matched": had_w_jet_match * ~had_b_jet_match,
    "unmatched": ~had_w_jet_match,
}

In [ ]:
np.sum(top_match_dict["top_matched"])

### Clustering and Lund plane splittings

In [ ]:
merged_top_events = presel_events[merged_top_jet_match]
had_top_jets = merged_top_events.FatJet[:, 0]

merged_ak8_pfcands = merged_top_events.FatJetPFCands
merged_ak8_pfcands = merged_ak8_pfcands[
    merged_ak8_pfcands.jetIdx == fatjet_idx[merged_top_jet_match]
]
merged_pfcands = merged_top_events.PFCands[merged_ak8_pfcands.pFCandsIdx]

In [ ]:
merged_pfcands_vector_ptetaphi = ak.Array(
    [
        [{kin_key: cand[kin_key] for kin_key in skim_vars} for cand in event_cands]
        for event_cands in merged_pfcands
    ],
    with_name="PtEtaPhiMLorentzVector",
)

In [ ]:
# jet definitions
dR = 0.8
cadef = fastjet.JetDefinition(fastjet.cambridge_algorithm, dR)
ktdef = fastjet.JetDefinition(fastjet.kt_algorithm, dR)

num_prongs = 3

In [ ]:
# cluster first with kT
kt_clustering = fastjet.ClusterSequence(merged_pfcands_vector_ptetaphi, ktdef)
kt_subjets = kt_clustering.exclusive_jets(num_prongs)
kt_subjets_pt = np.sqrt(kt_subjets.px**2 + kt_subjets.py**2)
kt_subjet_consts = kt_clustering.exclusive_jets_constituents(3)

# then re-cluster with CA
# won't need to flatten once https://github.com/scikit-hep/fastjet/pull/145 is released
ca_clustering = fastjet.ClusterSequence(ak.flatten(kt_subjet_consts, axis=1), cadef)
lds = ak.flatten(ca_clustering.exclusive_jets_lund_declusterings(1), axis=1)

In [ ]:
kt_subjets_vec = ak.zip(
    {"x": kt_subjets.px, "y": kt_subjets.py, "z": kt_subjets.pz, "t": kt_subjets.E},
    with_name="LorentzVector",
)

### Subjet matching and pT extrapolation uncertainty

In [ ]:
matching_dR = 0.2

In [ ]:
gen_quarks = ak.concatenate(
    [had_bs[merged_top_jet_match][:, :1], had_ws.children[merged_top_jet_match][:, :2]], axis=1
)

In [ ]:
gen_quarks

In [ ]:
sj_matched = []
sj_matched_idx = []

for i in range(num_prongs):
    sj_q_dr = kt_subjets_vec.delta_r(gen_quarks[:, i])
    sj_matched.append(ak.min(sj_q_dr, axis=1) <= matching_dR)
    sj_matched_idx.append(ak.argmin(sj_q_dr, axis=1))

In [ ]:
sj_matched = np.array(sj_matched).T
sj_matched_idx = np.array(sj_matched_idx).T

sj_matched_idx_mask = np.copy(sj_matched_idx)
sj_matched_idx_mask[~sj_matched] = -1
double_matched_event = np.any(
    [np.sum(sj_matched_idx_mask == i, axis=1) > 1 for i in range(3)], axis=0
).astype(int)
unmatched_quarks = np.sum(~sj_matched, axis=1)

### Calc SFs

In [ ]:
f = uproot.open("../corrections/ratio_feb20.root")

# 3D histogram: [subjet_pt, ln(0.8/Delta), ln(kT/GeV)]
ratio_nom = f["ratio_nom"].to_numpy()
ratio_nom_errs = f["ratio_nom"].errors()
ratio_nom_edges = ratio_nom[1:]
ratio_nom = ratio_nom[0]

zero_vals = ratio_nom == 0
ratio_nom[zero_vals] = 1
ratio_nom_errs[zero_vals] = 0

ratio_shape = f["ratio_nom"].values().shape

In [ ]:
max_pt_bin = ratio_nom_edges[0][-2]

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(ratio_nom[5].T[::-1], extent=[-1, 8, -5, 6.9], cmap="turbo")
plt.xlabel(r"ln$(0.8/\Delta)$")
plt.ylabel(r"ln$(k_T/GeV)$")
_ = plt.colorbar()

In [ ]:
# maxp = 0
# for i in range(ratio_shape[1]):
#     for j in range(ratio_shape[2]):
#         if len(pt_extrap_params[i][j]) > maxp:
#             maxp = len(pt_extrap_params[i][j])

# maxp

In [ ]:
pt_extrap_lookups = {"params": [], "errs": [], "sys_up_params": [], "sys_down_params": []}
max_fparams = 4  # max order (+1) of functions


def _np_pad(arr: np.ndarray, target: int = max_fparams):
    return np.pad(arr, ((0, target - len(arr))))


for i in range(ratio_shape[1]):
    for key in pt_extrap_lookups:
        pt_extrap_lookups[key].append([])

    for j in range(ratio_shape[2]):
        func = f["pt_extrap"][f"func_{i + 1}_{j + 1}"]
        pt_extrap_lookups["params"][-1].append(
            _np_pad(func._members["fFormula"]._members["fClingParameters"])
        )
        pt_extrap_lookups["errs"][-1].append(_np_pad(func._members["fParErrors"]))
        pt_extrap_lookups["sys_up_params"][-1].append(
            _np_pad(
                f["pt_extrap"][f"func_sys_tot_up_{i + 1}_{j + 1}"]
                ._members["fFormula"]
                ._members["fClingParameters"]
            )
        )
        pt_extrap_lookups["sys_down_params"][-1].append(
            _np_pad(
                f["pt_extrap"][f"func_sys_tot_down_{i + 1}_{j + 1}"]
                ._members["fFormula"]
                ._members["fClingParameters"]
            )
        )

for key in pt_extrap_lookups:
    pt_extrap_lookups[key] = dense_lookup(np.array(pt_extrap_lookups[key]), ratio_nom_edges[1:])

In [ ]:
n_sf_toys = 100
np.random.seed(42)
rand_noise = np.random.normal(size=[n_sf_toys, *ratio_nom.shape])

# produces array of shape ``[n_sf_toys, subjet_pt bins, ln(0.8/Delta) bins, ln(kT/GeV) bins]``
ratio_nom_smeared = ratio_nom + (ratio_nom_errs * rand_noise)
ratio_smeared_lookups = [
    dense_lookup(ratio_nom_smeared[i], ratio_nom_edges) for i in range(n_sf_toys)
]

In [ ]:
# save offsets to recover awkward structure later
ld_offsets = lds.kt.layout.offsets
flat_logD = np.log(0.8 / ak.flatten(lds).Delta).to_numpy()
flat_logkt = np.log(ak.flatten(lds).kt).to_numpy()
# repeat subjet pt for each lund declustering
flat_subjet_pt = np.repeat(ak.flatten(kt_subjets_pt), ak.count(lds.kt, axis=1)).to_numpy()

In [ ]:
high_pt_sel = flat_subjet_pt > max_pt_bin
hpt_logD = flat_logD[high_pt_sel]
hpt_logkt = flat_logkt[high_pt_sel]
hpt_sjpt = flat_subjet_pt[high_pt_sel]  # change this to 1/sjpt for next iteration

In [ ]:
# store polynomial orders for pT extrapolation
sj_pt_orders = np.array([np.power(hpt_sjpt, i) for i in range(max_fparams)]).T

In [ ]:
clip_max, clip_min = 10, 0.1
pt_lookup = pt_extrap_lookups["params"]
params = pt_lookup(hpt_logD, hpt_logkt)
pt_extrap_vals = np.maximum(np.minimum(np.sum(params * sj_pt_orders, axis=1), clip_max), clip_min)

In [ ]:
_ = plt.hist(
    np.log10(np.maximum(np.sum(params * sj_pt_orders, axis=1), 0.1)),
    np.linspace(-1, 1, 21),
    histtype="step",
)

In [ ]:
ratio_nom_vals = ratio_smeared_lookups[0](flat_subjet_pt, flat_logD, flat_logkt)

In [ ]:
ratio_nom_vals[high_pt_sel] = pt_extrap_vals

In [ ]:
sf_vals = []

# could be parallelised but not sure if memory / time trade-off is worth it
for i, ratio_nom_lookup in enumerate(ratio_smeared_lookups):
    ratio_nom_vals = ratio_nom_lookup(flat_subjet_pt, flat_logD, flat_logkt)
    reshaped_ratio_nom_vals = ak.Array(
        ak.layout.ListOffsetArray64(ld_offsets, ak.layout.NumpyArray(ratio_nom_vals))
    )
    sf_vals.append(
        np.prod(ak.prod(reshaped_ratio_nom_vals, axis=1).to_numpy().reshape(-1, num_prongs), axis=1)
    )

sf_vals = np.array(sf_vals)

In [ ]:
sf_vals

#### Old way of uncertainty calculation (not taking into account intra-jet correlations)

In [ ]:
# save offsets to recover awkward structure later
ld_offsets = lds.kt.layout.offsets
flat_logD = np.log(0.8 / ak.flatten(lds).Delta).to_numpy()
flat_logkt = np.log(ak.flatten(lds).kt).to_numpy()
# repeat subjet pt for each lund declustering
flat_subjet_pt = np.repeat(ak.flatten(kt_subjets_pt), ak.count(lds.kt, axis=1)).to_numpy()

In [ ]:
ratio_nom_vals = ratio_nom_lookup(flat_subjet_pt, flat_logD, flat_logkt)
ratio_nom_rel_errs = np.nan_to_num(
    (ratio_nom_errs_lookup(flat_subjet_pt, flat_logD, flat_logkt) / ratio_nom_vals) ** 2
)

reshaped_ratio_nom_vals = ak.Array(
    ak.layout.ListOffsetArray64(ld_offsets, ak.layout.NumpyArray(ratio_nom_vals))
)
reshaped_ratio_nom_rel_errs = ak.Array(
    ak.layout.ListOffsetArray64(ld_offsets, ak.layout.NumpyArray(ratio_nom_rel_errs))
)

sf_nom_vals = np.prod(
    ak.prod(reshaped_ratio_nom_vals, axis=1).to_numpy().reshape(-1, num_prongs), axis=1
)
sf_nom_errs = (
    np.sqrt(
        np.sum(
            ak.sum(reshaped_ratio_nom_rel_errs, axis=1).to_numpy().reshape(-1, num_prongs), axis=1
        )
    )
    * sf_nom_vals
)

In [ ]:
sf_nom_vals

In [ ]:
sf_nom_errs

In [ ]:
def get_ld_indices(vals, edges):
    indices = []

    for val, edge in zip(vals, edges):
        indices.append(
            np.clip(
                np.searchsorted(edge, val, side="right") - 1,
                0,
                len(edge) - 2,
            )
        )

    return indices

In [ ]:
# get lund plane bins
inds = get_ld_indices((flat_subjet_pt, flat_logD, flat_logkt), ratio_nom_edges)
# convert 3D indices to single scalar
scalar_inds = np.ravel_multi_index(inds, ratio_nom.shape)
reshaped_inds = ak.Array(ak.layout.ListOffsetArray64(ld_offsets, ak.layout.NumpyArray(scalar_inds)))
# reshaped_inds = pad_val(reshaped_inds, 40, -1, axis=1, clip=False)

In [ ]:
dups = 0
for row in reshaped_inds:
    _, counts = np.unique(row, return_counts=True)
    dups += np.sum(counts - 1)

print(f"Fraction of duplicate splittings: {dups / len(flat_logD):.2f}")